# Evaluating Environmental Predictors of Breeding Waterfowl Populations in the Central Interior Plateau of British Columbia 

## Adventures in machine-learning and regression-based population models
![image](overview_survey_area_extent.png)


### Overview - General Steps
1. Question - What is the distribution of species within the Central Interior Plateau? 
    * Identify main drivers
    * Identify areas of interest - diversity and abundance
    * Inform conservation planning
2. Data exploration and data wrangling
3. Identify analysis methods and tools
4. Perform analysis
5. Examine and refine


<a href="https://community.esri.com/groups/esri-training/blog/2018/10/19/use-the-five-step-gis-analysis-process"> <font size = -2>Modified from ESRI 5-Step GIS Analysis Process</font></a>


<style type="text/css">
.input_hidden{
    display: none
</style>

<style type="text/css"> # CSS hack to hide the cell input
.input_hidden{
    display: none


Issues tracking:

1. IRKernel loading packages from later version of R (3.5.3 instead of 3.5.1) - must change kernel spec as described here: https://github.com/IRkernel/IRkernel/issues/183


2. jupyterlab extension to close cells * requires installation of nodejs and npm 
    conda install nodejs
    conda install npm 
   
    - required update of pip
    - which required adding forge channel first:
        conda config --add channels conda-forge 
        conda update pip
     now
        conda install nodejs
        conda install npm
     but required update of core-js with 
         npm install --save core-js@^3
     
     then follow: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html and run
         conda install -c conda-forge jupyter_contrib_nbextensions
     then 
         jupyter contrib nbextension install --sys-prefix # for system wide esp. useful in virtual env
     then enable the path but first list available Jupyter paths 
         jupyter --paths

     Check jupyterlab extensions with 
        jupyter nbextension list
    
    }
</style>


In [ ]:
# Housekeeping - manage packages 
version
Packages <- c("dplyr", "stringr", "ggplot2", "gganimate", "RColorBrewer", "reshape2", "gridExtra", "tidyverse",
             "repr", "knitr")
# # Install packages if not installed 
# if (!require("pacman")) install.packages("pacman")
#     pacman::p_load(Packages)

lapply(Packages, require, character.only=TRUE)
(.packages)() # List loaded packages

Here is a look at the survey data--the head and tail views of the data frame.

In [ ]:
setwd("C:/Users/hashimotoy/Documents/MGIS/Documentation/Reports/Jupyter")
df <- read.csv("iws_pts_filtered_20190620.csv" )

head(df)
tail(df)
nrow(df)

### Data Exploration - Species Observation Records

In [ ]:
n_sp <- unique(df$sp_id)

sp <- df %>%
    group_by(sp_id) %>%
    summarise(Population = sum(pop))

sp_top_5 <- sp %>%
    arrange(desc(Population))%>%
    top_n(5)

sp_bot_5 <- sp %>%
    arrange(Population)%>%
    top_n(-5)

sp_top_10 <- sp %>%
    arrange(desc(Population))%>%
    top_n(10)

print(paste("Total distinct species of waterfowl : ",length(n_sp)))
writeLines("\n5 MOST common species:")
sp_top_5
writeLines("\n5 LEAST common species:") 
sp_bot_5   



What are the top 15 most common species? 

In [ ]:
sp <- df %>% group_by(sp_id) %>% summarise(Population = sum(pop)) %>% arrange(desc(Population)) %>% 
    top_n(15)
# sp
p <- ggplot(data = sp, aes(x = reorder(sp_id, -Population), y = Population)) + geom_bar(stat = "identity", 
    fill = "gray40") + ggtitle("Cumulative Indicated Breeding Population", subtitle = "Top 15 Most Abundant Species 2007-2017") + 
    xlab("Species") + geom_text(aes(label = Population), size = 3, hjust = 0.5, vjust = -0.4) + 
    theme(plot.title = element_text(hjust = 0.5), plot.subtitle = element_text(hjust = 0.5), 
        axis.text.x = element_text(angle = 45, vjust = 0.5))
options(repr.plot.width = 6.5, repr.plot.height = 4)
p

What is the distribution of species within ecosections?

In [ ]:
eco_sp <- df %>% group_by(eco) %>% summarise(Population = sum(pop)) %>% arrange(desc(Population))

# Create a custom colour-blind-friendly scale
library(RColorBrewer)
cbp1 <- c("#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", 
    "#CC79A7")

names(cbp1) <- levels(eco_sp$eco)
colFill <- scale_fill_manual(names(cbp1), values = cbp1)
colColour <- scale_colour_manual(name = "ECO", names(cbp1), values = cbp1)

g <- ggplot(eco_sp, aes(x = reorder(eco, -Population), y = Population, fill = eco)) + 
    geom_bar(stat = "identity") + ggtitle("Total Cumulative Indicated Breeding Population by Ecosection \n 2007-2017") + 
    xlab("Ecosection") + labs(color = "Ecosection") + colFill + geom_text(aes(label = Population), 
    size = 2.5, hjust = 0.5, vjust = -0.5) + theme(axis.text.x = element_text(angle = 45, 
    vjust = 0.5), plot.title = element_text(hjust = 0.5), legend.position = "none")
options(repr.plot.width = 6, repr.plot.height = 4)
g

The Cariboo Basin (CAB) is clearly a very important Ecosection for breeding waterfowl populations especially when taking the Ecosection area into account as visualized by population density below.

In [ ]:
eco_area <- read.csv("eco_transect_area_summary.csv")
eco_p <- merge(eco_sp, eco_area)
eco_p$p_density <- eco_p$Population/eco_p$area_surveyed*10000

g <- ggplot(eco_p, aes(x=reorder(eco, -p_density),y=p_density, fill=eco)) + geom_bar(stat="identity") + 

    ggtitle(label="Cumulative Indicated Breeding Population Density", subtitle="by Ecosection 2007-17") + colFill +
    geom_text(aes(label=round(p_density, digits=0)), size=3, hjust=0.5, vjust=-0.5) +
    xlab("Ecosection") + ylab("Breeding Population per Hectare") + 
    theme(axis.text.x = element_text(angle=45,vjust=.5), 
        plot.title = element_text(hjust=0.5),
        plot.subtitle = element_text(hjust=0.5), 
        legend.position = "none")
options(repr.plot.width=6,repr.plot.height=4)
g


How does the population density vary through time?

In [ ]:
eco_yr <- df %>% group_by(eco, year_) %>% summarise(pop = sum(pop)) %>% left_join(., 
    eco_area, by = "eco") %>% mutate(density = pop/area_surveyed * 10000)

shp1 <- c(5,6,8,15:19)
colShape <- scale_shape_manual(name = "ECO", names(cbp1), values = shp1)

# #Plot the outcome
g <- ggplot(eco_yr, aes(year_, density, colour = eco, shape = eco)) + ggtitle("Population Density in Ecosection by Year", 
    subtitle = "All Waterfowl") + geom_point() + geom_line(size = 0.5) + colShape + 
    colColour + theme(legend.position = "bottom", legend.direction = "horizontal", 
    legend.box = "vertical", axis.text.x = element_blank(), plot.title = element_text(hjust = 0.5), 
    plot.subtitle = element_text(hjust = 0.5)) + xlab("Ecosection") + ylab("Total Breeding Population Density per ha")
options(repr.plot.width = 6, repr.plot.height = 4)
g

Temporal animations can provide a different perspective. Animated lines, dots and bar charts below. I got carried away obviously. 

In [ ]:
## 'gganimate' Line chart with dots using 'transition_reveal'
g <- ggplot(eco_yr, aes(year_, density, group = eco, shape = eco, colour = eco)) + labs(title = "Population Density in Ecosection by Year") + 
    geom_point(size = 4) + geom_line(size = 1) + colShape + colColour + theme(legend.position = "bottom", 
    axis.text.x = element_text(angle = 45, vjust = 0.5), plot.title = element_text(hjust = 0.5), 
    plot.subtitle = element_text(hjust = 0.5, size = 17)) + scale_size(guide = FALSE) + 
    xlab("Ecosection") + ylab("Total Breeding Population Density per ha")
g + geom_point((aes(group = seq_along(year_), size = 2))) + transition_reveal(year_) + 
    labs(subtitle = "{frame_along}")

## Dot plot with 'shadow_wake' and 'transition_time'
g <- ggplot(eco_yr, aes(eco, density, size = density, colour = eco)) + geom_point(size = 9) + 
    colColour + ggtitle(label = "Population Density in Ecosection by Year") + theme(legend.position = "bottom", 
    axis.text.x = element_text(angle = 45, vjust = 0.5, size = 9), plot.title = element_text(hjust = 0.5), 
    plot.subtitle = element_text(hjust = 0.5, size = 17)) + xlab("Ecosection") + 
    ylab("Total Breeding Population Density per ha")
g + transition_time(year_) + labs(subtitle = "{frame_time}") + shadow_wake(wake_length = 0.2, 
    alpha = FALSE)

## Bar chart with 'transition_time'
g <- ggplot(eco_yr, aes(eco, density, fill = eco)) + geom_bar(stat = "identity") + 
    colFill + theme(legend.position = "bottom", legend.direction = "horizontal", 
    legend.box = "vertical", axis.text.x = element_text(angle = 45, vjust = 0.5, 
        size = 9), plot.title = element_text(hjust = 0.5), plot.subtitle = element_text(hjust = 0.5, 
        size = 17)) + xlab("Ecosection") + ylab("Total Breeding Population Density per ha")
g + transition_time(year_) + labs(title = "Population Density in Ecosection by Year", 
    subtitle = "{frame_time}")

![SegmentLocal](popd_eco_yr_line.gif "segment")

![SegmentLocal](popd_eco_yr_point.gif "segment")

![SegmentLocal](popd_eco_yr_bar.gif "segment")

Let's drill down to the species distributions within Ecosections focusing on the top 10 most abundant.

In [ ]:
sp_10 <- factor(sp_top_10$sp_id) # Unless factor original 31 levels are retained
sp_10
sp_10 <- tolower(sp_10) 

In [ ]:
# Load the processed data with associated environmental variables summarized to grid cell.

library(tidyr)
library(gridExtra)
library(tidyverse)
library(repr) # Plot sizes in Jupyter wrapper for R 
iws_wide <- read.csv("merged_id1_400_years.csv")
# Note processed data have sp codes in lowercase in wide format for model input
iws_long <- gather(iws_wide, sp_id, pop, amwi:ground) 

eco_sp_yr <- iws_long %>%
    group_by(eco, year_,sp_id)%>%
    summarise(Population = sum(pop))%>%
    left_join(., eco_area, by = "eco")%>%
    mutate(density = Population/area_surveyed*10000)%>%
    filter(sp_id %in% sp_10)

head(eco_sp_yr)

Visualize and compare distributions of the top most common species

In [ ]:
bp <- ggplot(eco_sp_yr, aes(x=eco, y=density, colour=eco)) + 
    geom_boxplot() + colColour + ggtitle(label="Relative Population Density Distribution", subtitle="Top 10 most abundant species in Study Area") +
    theme(legend.position="bottom",legend.direction = "horizontal", legend.box="vertical", 
            axis.text.x = element_blank(), 
              plot.title = element_text(hjust = 0.5),
            plot.subtitle = element_text(hjust=0.5, size=9)) + 
    ylab("Total Breeding Population Density per ha") + xlab("Ecosection") +
    stat_summary(fun.y=mean, geom="point", shape=5, size=2)# Add means

options(repr.plot.width=7,repr.plot.height=5)
bp + facet_wrap(. ~ toupper(sp_id), ncol=10)

# Display with dynamic scale ranges
options(repr.plot.width=8,repr.plot.height=13)
bp + facet_wrap(. ~ toupper(sp_id), ncol=2, scales="free") + geom_point(size=0.4)



In [ ]:
sp_yr <- ggplot(eco_sp_yr, aes(x = year_, y = density, colour = eco, shape = eco)) + 
    geom_point() + geom_smooth(method = "auto", se = FALSE, size = 0.7) + colShape + 
    colColour + ggtitle(label = "Species Distributions", subtitle = "2007 to 2017") + 
    theme(legend.position = "bottom", legend.direction = "horizontal", legend.box = "vertical", 
        plot.title = element_text(hjust = 0.5), plot.subtitle = element_text(hjust = 0.5, 
            size = 9)) + ylab("Total Breeding Population Density per ha") + xlab("Year (2007-2017)")

# Display with dynamic scale ranges
options(repr.plot.width = 8, repr.plot.height = 13)
sp_yr + facet_wrap(. ~ toupper(sp_id), ncol = 2, scales = "free") + geom_point(size = 0.4)

Table view of the Top 10 most abundant species within each Ecosection

In [ ]:
# Get the top 5 most common species in each ecosection
# sp_id contains species guilds as well as single species
sp_long <- iws_wide[,1:grep("wodu", colnames(iws_wide))] %>%
    gather(sp_id, pop, amwi:wodu) %>%
    group_by(eco, sp_id)%>%
    summarise(pop=sum(pop))%>%
    group_by(eco)%>%
    mutate(ranks = dense_rank(desc(pop)))%>%
    filter(ranks<=10)%>%
    select(-c(pop))%>%
    spread(eco, sp_id) %>%
    group_by(ranks)

print("Top 10 Cumulative Most Abundant Species by Ecosection")
# function to convert lowercase to upper
data.frame(lapply(sp_long, function(v) {
  if (is.character(v)) return(toupper(v))
  else return(v)
}))



In [ ]:
# Animation of facet_grid scatterplot - too dizzying really.

g <- ggplot(eco_sp_yr, (aes(x = eco ,y = density, size = density, colour=eco))) +
    ggtitle("Population Density in Ecosection by Year") +
      geom_point() + colColour + theme(legend.position="bottom",legend.direction = "horizontal", legend.box="vertical", 
                                  axis.text.x = element_blank(), 
                                    plot.title = element_text(hjust = 0.5),
                                    plot.subtitle = element_text(hjust=0.5, size=17))+ 
       xlab("Ecosection") + ylab("Total Breeding Population Density per ha")


g + facet_wrap(. ~ toupper(sp_id), ncol=3) + transition_time(year_) + labs(subtitle = '{frame_time}')

## Evaluation of Modelling Approaches and Data Inputs

Points of consideration for the modelling framework, data inputs, and their interactions

#### Data Properties
* Accuracy and uncertainty
* Data acquisition and availability
* Logical consistency and timeliness
* Scale, extent and resolution 
* Direct vs Indirect 
* Mechanisitic vs correlated 
* Ecological and physiological

####  Model Properties
* Predictive vs Explanatory (Random Forest and Regression-based approaches)
* Intepretation - scale of analyses and applications

<img src="modeling_guisan.png" alt="Drawing"/>


## Waterfowl Survey Observation data
##### Weaknesses
* Positional uncertainty of observation data
* Inconsistency of survey methods - technological change
* Survey conditions (climate and weather-dependent, pilot experience)
* Relative coverage - 400m wide strip transects spaced 10 miles apart (~2.5% study area)
* Inconsistent with methods of North American Breeding Waterfowl Habitat and Population Survey (1955-present)

##### Strengths
* Consistent and highly experienced observers 
* Continuous, annual basis since 2006 
* Large database ~ 30,000+ 
* Large extent ~ 10,000,000+ ha (~40,000+ sq miles)


<img src="overview_observations.png" alt="Drawing" style="width: 400px;"/>

## Explanatory Predictor Candidates
#### Climate
* Growing Degree Days - Primary Productivity
* Precipitation as Snow - Hydrology is driven by snowpack 
* Spring Temperature - Timing of freshet
* Heat Moisture Index - (MAT+10)/(MAP/1000) 

#### Landscape
* Topography - aspect, slope, elevation
* Landcover - readily available interpreted data product (CEC 2010)
* Disturbance - roads atlas, fire database, Mountain Pine Beetle 
* Wetland habitat - lakes, rivers, wetlands
    * smaller wetlands <10 ha ≈ increased productivity
    * data creation driven by forestry industry
    
#### Land management
* Agricultural Land Reserve
* Protected Areas
* Tenure

<sub>See 'data_dictionary.xlsx'</sub>

<< PLACEHOLDER PREDICTOR SURFACES 

## Random Forest - Overview of steps
1. Random forest approach selected for predictive power
2. Tool selection
    * 'cforest' party package - conditional inference trees
    * 'randomForest' - favours continuous variables and variables with many categories
    * 'Forest-based Classification and Regression' ArcGIS Pro v2.2+
3. Variable selection
4. Data processing
5. Model evaluation
     * Model stability
     * Model accuracy

<a href=https://core.ac.uk/download/pdf/12170130.pdf><div style="text-align: right"> <font size = -1>Strobl et al, 2009</font> </div></a>  

![image](random_forest_diagram_complete_vankar_jaganath_cc.png) 
<div style="text-align: right"> <sub> cc Vankar Jaganath <sub> </div>

### Variable Selection 
##### Ecological considerations and issues
* Climate normals vs mean vs timeliness - response lag
* Species breeding philopatry
* Density-dependence and interspecific competition
* Wintering conditions
##### Addressing correlation, collinearity and variance inflation
* High degree of correlation in climate and ecology 
* Imbalanced classes of predictor variables
* Spatial variability
* *Scale



#### Example collinearity - Climate
* Correlation thresholds:
    * < 0.7 (Green, 1979) 
    * < 0.8 (Menard, 2002)
* Multicollinearity - correlation between 3+ variables even when pair-wise correlation is absent
    * VIF threshold < 5  
    * critical threshold 5-10 (Guisan et al, 2017)


In [ ]:
## Address Correlations, Multicollinearity and VIF
Packages <- c('Hmisc', 'corrplot', 'car')
lapply(Packages, require, character.only=TRUE)
iws_wide <- read.csv("id1_400_years.csv")
# Example climate variable selection
# pairwise correlations with 'rcorr'
clim_var <- iws_wide[,grep("mean_ahm", colnames(iws_wide)):grep("mean_shm", colnames(iws_wide))]
# Set plot size
options(repr.plot.width=8, repr.plot.height=8)
#Check correlations
corr <- rcorr(as.matrix(clim_var))
corrplot(corr$r, method='ellipse', number.cex= 10/ncol(df))


Filter candidates identifying key drivers: snowpack, growing degree days (DD5), moisture index

In [ ]:
library(dplyr)
# Subset
# grep("mean_shm", colnames(iws_wide))
# grep("mean_pas_wt", colnames(iws_wide))
# grep("mean_dd5", colnames(iws_wide))

clim_var_1 <- iws_wide[,c(76, 77, 67)]

# Set plot size
options(repr.plot.width=8, repr.plot.height=5)

# Check correlations
# Set plot size
corr <- rcorr(as.matrix(clim_var_1))
corrplot(corr$r, method='number', number.cex= 10/ncol(df))
str(clim_var_1)


![image](corr_clim_ellipse.png)
![image](corr_landscape_ellipse.png)

### <<PLACEHOLDER FOR FILTERED PREDICTORS

In [ ]:
# Assess multicollinearity
library(usdm)

v <- vif(clim_var_1)
vm <- v$VIF
names(vm) <- v$Variables
str(vm)
dotchart(vm[order(vm)], cex = 0.6, pch = 19,
         gcolor = "#999999",
         xlab = "varImp")

###  < PLACEHOLDER cforest model run example

### Random Forest Model Evaluation
cforest_unbiased
    * can handle correlated predictors 
    * computationally intensive 
    * preliminary model runs without filtering correlated variables ~ 240+ hours
##### Model Stability - Variable Important Measures (varimp)
* Subset data into training, validate and test subsets (70, 30, 30%)
* Run models with different seeds
* Compare ranks of varimp ('SuperRanker') 
* Evaluate 

![image](varimp_multi.png)


<sub>Toggle cell below for model evalution code - note raw data not included in notebook<sub>

In [ ]:
# Sequential rank agreement methods for comparison of ranked lists
# https://github.com/tagteam/SuperRanker

fgcRequire <- function(pkg) {
  if(pkg %in% rownames(installed.packages()) == FALSE) {
    install.packages(pkg)
  }
}

fgcRequire("devtools")
library(devtools)

install_github('TagTeam/SuperRanker')
library(SuperRanker)  # For the SuperRanker package
wd <- "../eco_CAB/"
wdA <- paste0(wd,"results-seed-47")
wdB <- paste0(wd,"results-seed-742938")
wdC <- paste0(wd,"results-seed-42")
              
af <- list.files(wdA, "*.csv$")
bf <- list.files(wdB, "*.csv$")
cf <- list.files(wdB, "*.csv$")

if (length(intersect(af,bf)) != length(bf)) { 
  print("unpaired input sets")
  return()
}

f <-  af[1] # DEBUG

err <- as.data.frame(sapply(af, 
#tErr <- sapply(af, 
function(f) {
  setwd(wdA)
  a <- read.csv(f)
  sa <- order(a$vi, decreasing = TRUE)
  
  setwd(wdB)
  b <- read.csv(f)
  sb <- order(b$vi, decreasing = TRUE)
  
  setwd(wdC)
  c <- read.csv(f)
  sc <- order(b$vi, decreasing = TRUE)
  

  i <- cbind(sa,sb,sc)
  mlist <- matrix(i,ncol=3)
  #rank <- overlap(mlist)
  rank <- smooth_sra(mlist)
  
  print(paste("Result:", f))
  #print(rank)
  #print(rank)
  ul <- cbind(rank$lower,rank$upper)
  
  #r <- ul[1,] # DEBUG
  e <- apply(ul, 1, function(r) {
    return(abs(r[1] - r[2]))
  })
  
  print(paste("Error:"))
  print(e)
  
  m <- mean(e)
  return(c(m,e))
})
)


### < PLACEHOLDER Refined model after assessing VARIMP filtering 

### <<PLACEHOLDER Accuracy Assessment

### << PLACEHOLDER Model Predictions

![image](cab_layout_MALL.png)
![image](cab_layout_BAGO.png)
![image](cab_layout_LESC.png)

## Regression-based  - Overview of steps
1. Generalized Linear Mixed Models approach
    * Allows for variable response distributions - Poisson, Gamma, etc
    * Zero-inflation (16ha spatial and yearly scale ~ 92% records recorded 0)
    * Fixed and random effects
    * Flexible, efficient
2. Tool selection
    * R packages: lme4, glmmTMB
3. Variable selection 
    * Identified in Random Forest modeling - but collinearity MUST be addressed
4. Data processing
    * Identify response distribution 
    * Standardization 
5. Model evaluation
     * Model stability
     * Model accuracy

  


In [ ]:
iws_wide

g <- ggplot(iws_wide, )

### Variable Selection methods???

## Tools and technologies

* Jupyter notebook - computational notebook 
* tidyverse - family of R packages sharing a common syntax (dplyr, ggplot2, purr, tidyr, etc) 
* rpy2 - interface to R from within Python - switch between R and Python with R magics 
* R-ArcGIS Bridge - interface ArcGIS and R (R v 3.2.2+, Python Script Toolbox
* R-markdown
* Python Toolbox -  